# Sample training script with naive competitive self-play.

### Switch directory in Google drive so as to import CDA env.


In [1]:
is_colab = True
is_1st_run = False

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# %cd "/root/ray_results/"
# !ls -l
# #!rm -rf PPO_continuousDoubleAuction-v0_*
# !ls -l
# !pwd

# %cd "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/"
# !ls -l

# #!pip install -r requirements.txt

# #!pip install tensorflow==2.2.0
# #!pip install ray[rllib]==0.8.5

# #!pip show tensorflow
# #!pip show ray

# #!pip freeze > requirements.txt

In [3]:
if is_colab == False and is_1st_run == True:
    !pip install sortedcontainers
    !!pip install scikit-learn
    !pip install tabulate
    !pip install matplotlib

In [4]:
# !pip install -U ipywidgets
# !pip install pettingzoo

In [5]:
# !pwd

In [6]:
# os.chdir('..')

In [7]:
# !pip install -e.

In [8]:
# !pip uninstall continuousDoubleAuction
# !pip uninstall continuousDoubleAuction-v0

In [9]:
# !pip show continuousDoubleAuction
# !pip show continuousDoubleAuction-v0

In [10]:
# os.chdir('gym_continuousDoubleAuction')

In [11]:
# !pwd

In [12]:
if is_colab == True:
    !pip install -U ray[rllib]==2.48.0

    from google.colab import drive
    drive.mount('/content/gdrive')

    %cd 'gdrive/MyDrive/Colab Notebooks/MARL/gym-continuousDoubleAuction'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/MARL/gym-continuousDoubleAuction


In [13]:
import ray
import ray.rllib
import ray.tune

print("Ray version:", ray.__version__)

Ray version: 2.48.0


In [14]:
!pip show gymnasium

Name: gymnasium
Version: 1.0.0
Summary: A standard API for reinforcement learning and a diverse set of reference environments (formerly Gym).
Home-page: https://farama.org
Author: 
Author-email: Farama Foundation <contact@farama.org>
License: MIT License
Location: /usr/local/lib/python3.12/dist-packages
Requires: cloudpickle, farama-notifications, numpy, typing-extensions
Required-by: dopamine_rl


### Imports

In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['RAY_DEBUG_DISABLE_MEMORY_MONITOR'] = "True"
os.environ['PYTHONWARNINGS'] = 'ignore::DeprecationWarning'

import argparse

# import gym
import gymnasium as gym

import random
import numpy as np

from collections import defaultdict
from typing import Dict

import ray
from ray import tune
from ray.rllib.utils import try_import_tf
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.policy import Policy

from ray.rllib.algorithms.ppo import PPOConfig

from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.sample_batch import SampleBatch

from ray.rllib.algorithms.callbacks import DefaultCallbacks

from ray.tune.logger import pretty_print

import sys
if "../" not in sys.path:
    sys.path.append("../")


from gym_continuousDoubleAuction.envs.continuousDoubleAuction_env import continuousDoubleAuctionEnv

from gym_continuousDoubleAuction.train.model.model_handler import CustomRLModule

from gym_continuousDoubleAuction.train.policy.policy_handler import (
    # make_RandomPolicy,
    # gen_policy,
    # set_agents_policies,
    # create_train_policy_list,
    create_multi_agent_config,
    policy_mapping_fn,
    # create_and_train_algorithm,
)
from gym_continuousDoubleAuction.train.weight.weight_handler import (
    get_trained_policies_name, get_max_reward_ind, cp_weight)
from gym_continuousDoubleAuction.train.storage.store_handler import storage



from gym_continuousDoubleAuction.train.callbk.callbk_handler import store_eps_hist_data
from gym_continuousDoubleAuction.train.callbk.self_play_callback_mod import SelfPlayCallback



from gym_continuousDoubleAuction.train.logger.log_handler import (
    create_dir, log_g_store, load_g_store)
from gym_continuousDoubleAuction.train.plotter.plot_handler import (
    plot_storage, plot_LOB_subplot, plot_sum_ord_imb, plot_mid_prices)
from gym_continuousDoubleAuction.train.helper.helper import (
    ord_imb, sum_ord_imb, mid_price)


print(f'Imports all OK.')

Imports all OK.


### Global


In [16]:
# CDA_env args
num_agents = 4
num_trained_agent = 2 #
num_policies = num_agents # Each agent is using a separate policy
num_of_traders = num_agents
tape_display_length = 10
tick_size = 1
init_cash = 1000000
max_step = 4096 # per episode, -1 in arg. (~7.2s/1000steps/iter)
is_render = False

# RLlib config
# train_policy_list = create_train_policy_list(num_trained_agent, "policy_")
#num_cpus = 0.25
num_gpus = 0.75 #0
num_cpus_per_worker = 0.25
num_gpus_per_worker = 0
num_workers = 2
num_envs_per_worker = 4
batch_mode = "complete_episodes"
rollout_fragment_length = 128
train_batch_size = max_step
sgd_minibatch_size = 256
num_iters = 2

# log_base_dir = "/content/gdrive/My Drive/Colab Notebooks/gym-continuousDoubleAuction/results/"
log_base_dir = "results/"
log_dir = log_base_dir + "ray_results/"

# Chkpt & restore
local_dir = log_base_dir + "chkpt/"
chkpt_freq = 10
chkpt = 320
restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)
is_restore = True # True / False

# log & load
log_g_store_dir = log_base_dir + "log_g_store/"
create_dir(log_base_dir)
create_dir(log_g_store_dir)

# Environment configuration
env_config = {
    "num_of_agents": num_agents,
    "init_cash": 10000,
    "tick_size": 1,
    "tape_display_length": 10,
    "max_step": 1000,
    "is_render": False
}

# get obs & act spaces from dummy CDA env
# single_CDA_env = continuousDoubleAuctionEnv(
#     num_of_traders,
#     init_cash,
#     tick_size,
#     tape_display_length,
#     max_step,
#     is_render)
single_CDA_env = continuousDoubleAuctionEnv(env_config)
obs_space = single_CDA_env.get_observation_space(single_CDA_env.agents[0])
act_space = single_CDA_env.get_action_space(single_CDA_env.agents[0])
print(single_CDA_env.agents)  # Should be a non-empty list
print(single_CDA_env.get_observation_space(single_CDA_env.agents[0]))  # Should return a valid gym.Space
print(single_CDA_env.get_action_space(single_CDA_env.agents[0]))  # Should return a valid gym.Space

def env_creator(env_config):
    return continuousDoubleAuctionEnv(env_config)

# Register environment with ray.tune - this is the key fix!
tune.register_env("continuousDoubleAuction-v0", env_creator)

# register custom model (neural network)
ModelCatalog.register_custom_model("model_disc", CustomRLModule)

ray.shutdown()
# start ray
ray.init(
    ignore_reinit_error=True,
    log_to_driver=True,
    num_cpus=2,
    dashboard_host="127.0.0.1",  # replaces webui_host
    dashboard_port=8265,          # default port; replaces webui_port
    # include_dashboard=True,        # default True
    include_dashboard=False,        # default True

)

# Global storage, a ray actor that run on it's own process & it needs to be declared after ray.init().
g_store = storage.options(name="g_store", lifetime="detached").remote(num_agents)

Folder creation failed or folder already exists: results/
Folder creation failed or folder already exists: results/log_g_store/
get_action_space, agent_id: agent_2
get_action_space, self.action_space[agent_id]: Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))
['agent_2', 'agent_0', 'agent_1', 'agent_3']
Box(-inf, inf, (40,), float32)
get_action_space, agent_id: agent_2
get_action_space, self.action_space[agent_id]: Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))
Tuple(Discrete(3), Discrete(4), Box(-1.0, 1.0, (1,), float32), Box(0.0, 1.0, (1,), float32), Discrete(12))


2025-08-25 02:01:19,107	INFO worker.py:1927 -- Started a local Ray instance.


(raylet) It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="12fcccfd-2430-49c9-a489-ea782635f9f9", ...)


### Policies

In [17]:
policies, policies_to_train = create_multi_agent_config(
    obs_space, act_space, num_agents, num_trained_agents=num_trained_agent)

policies: {'policy_0': <ray.rllib.policy.policy.PolicySpec object at 0x78d2ac6d8500>, 'policy_1': <ray.rllib.policy.policy.PolicySpec object at 0x78d288733ce0>, 'policy_2': <ray.rllib.policy.policy.PolicySpec object at 0x78d288733d10>, 'policy_3': <ray.rllib.policy.policy.PolicySpec object at 0x78d288733ad0>}
policies_to_train: ['policy_0', 'policy_1']


### RLlib config

In [18]:
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig

def get_config():
    config = (
        PPOConfig()
        .environment(
            "continuousDoubleAuction-v0",
            # continuousDoubleAuctionEnv,
            # env_config={
            #     "num_of_agents": num_of_traders,
            #     "init_cash": init_cash,
            #     "tick_size": tick_size,
            #     "tape_display_length": tape_display_length,
            #     "max_step": max_step - 1,
            #     "is_render": is_render,
            # }
            env_config=env_config,
            # env_config={"disable_env_checker": True},
        )
        .multi_agent(
            policies=policies,
            policy_mapping_fn=policy_mapping_fn,
            policies_to_train=policies_to_train,
        )
        # .training(
        #     model={
        #         "custom_model": CustomLSTMRLModule,
        #         # "custom_model_config": {
        #         #     "fcnet_hiddens": [256, 256],  # Neural network architecture
        #         #     "fcnet_activation": "relu",
        #         # },
        #     }
        # )
        .env_runners(
            # num_env_runners=num_workers,
            num_env_runners=0,
            # num_envs_per_env_runner=num_envs_per_worker,
            # rollout_fragment_length=rollout_fragment_length,
            # batch_mode=batch_mode,
        )
        .learners(
            num_learners=0,  # Typically 1 learner unless using distributed training
            # num_gpus_per_learner=num_gpus,  # Trainer GPU allocation
            # num_cpus_per_learner=num_cpus_per_worker,
        )
        .training(
            train_batch_size_per_learner=train_batch_size,
            train_batch_size=1000,
            num_epochs=5,
        )
        .callbacks(SelfPlayCallback)
        # .output_dir(log_dir)
        .framework("torch")  # Explicitly set framework if needed
        .debugging(log_level="DEBUG")
        # .api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)
    )

    # # Optional: Configure resources more granularly if needed
    # if num_gpus_per_worker > 0:
    #     config.env_runners(
    #         num_gpus_per_env_runner=num_gpus_per_worker
    #     )

    return config

### Train


In [19]:
def go_train(config):
    # trainer = ppo.PPOTrainer(config=config, env="continuousDoubleAuction-v0")
    # algo = config.build()
    algo = config.build()

    # if is_restore == True:
    #     trainer.restore(restore_path)

    # g_store = ray.util.get_actor("g_store")
    # result = None
    for i in range(num_iters):
        result = algo.train()

    #     print(pretty_print(result)) # includes result["custom_metrics"]
    #     print("training loop = {} of {}".format(i + 1, num_iters))
    #     print("eps sampled so far {}".format(ray.get(g_store.get_eps_counter.remote())))

    #     if i % chkpt_freq == 0:
    #         checkpoint = algo.save(local_dir)
    #         print("checkpoint saved at", checkpoint)

    # checkpoint = algo.save(local_dir)
    # print("checkpoint saved at", checkpoint)
    # print("result['experiment_id']", result["experiment_id"])

    # return result["experiment_id"]
    return None

# run everything
experiment_id = go_train(get_config())

2025-08-25 02:01:22,163	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-08-25 02:01:22,191	WARNING algorithm_config.py:5033 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html


set_agg_LOB:flattened: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
set_agg_LOB:flattened: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
set_agg_LOB:flattened: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
set_agg_LOB:flattened: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
reset (observations): {'agent_0': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32), 'agent_1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 

2025-08-25 02:01:23,274	INFO connector_pipeline_v2.py:272 -- Added AddObservationsFromEpisodesToBatch to the end of EnvToModulePipeline.
2025-08-25 02:01:23,615	INFO connector_pipeline_v2.py:272 -- Added AddTimeDimToBatchAndZeroPad to the end of EnvToModulePipeline.
2025-08-25 02:01:24,013	INFO connector_pipeline_v2.py:272 -- Added AddStatesFromEpisodesToBatch to the end of EnvToModulePipeline.
2025-08-25 02:01:24,344	INFO connector_pipeline_v2.py:272 -- Added AgentToModuleMapping to the end of EnvToModulePipeline.
2025-08-25 02:01:24,624	INFO connector_pipeline_v2.py:272 -- Added BatchIndividualItems to the end of EnvToModulePipeline.
2025-08-25 02:01:25,045	INFO connector_pipeline_v2.py:272 -- Added NumpyToTensor to the end of EnvToModulePipeline.
2025-08-25 02:01:25,056	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=

Streaming output truncated to the last 5000 lines.
do_actions -> actions: [{'ID': 'agent_1', 'side': None, 'type': 'market', 'size': 8.0, 'price': -1.0}, {'ID': 'agent_0', 'side': 'ask', 'type': 'limit', 'size': 500.0, 'price': 2.0}, {'ID': 'agent_2', 'side': None, 'type': 'market', 'size': 51.0, 'price': -1.0}, {'ID': 'agent_3', 'side': None, 'type': 'market', 'size': 51.0, 'price': -1.0}]

Order NOT approved: -ve NAV for trader_ID 0.

set_agg_LOB:flattened: [ 181.   89.  373.  350.  320.  127.  208.  501.    0.    0.   18.   17.
   15.   13.    5.    4.    3.    1.    0.    0. -450. -232. -500. -697.
  -99. -501.    0.    0.    0.    0.  -43.  -63.  -72.  -81.  -82.  -83.
    0.    0.    0.    0.]
on_episode_step
set_agg_LOB:flattened: [ 181.   89.  373.  350.  320.  127.  208.  501.    0.    0.   18.   17.
   15.   13.    5.    4.    3.    1.    0.    0. -450. -232. -500. -697.
  -99. -501.    0.    0.    0.    0.  -43.  -63.  -72.  -81.  -82.  -83.
    0.    0.    0.    0.]
{'agent

### Plot all steps.

Agt_0, 1, 2 are trained agents (with PPO) while the rest are random agents.

In [20]:
# plot_storage(num_agents, init_cash)

In [21]:
# plot_storage(num_agents, init_cash, "step", "NAV")

In [22]:
# plot_storage(num_agents, init_cash, "step", "num_trades")

### Log/load last episode

In [23]:
# log_g_store(log_g_store_dir, num_agents, experiment_id)
# load_g_store(log_g_store_dir, num_agents, experiment_id)

### Plot steps from last episode.

In [24]:
# plot_storage(num_agents, init_cash)

In [25]:
# plot_storage(num_agents, init_cash, "step", "NAV")

In [26]:
# plot_storage(num_agents, init_cash, "step", "num_trades")

### LOB from last episode.

In [27]:
# g_store = ray.util.get_actor("g_store")
# #store = ray.get(g_rere.get_storage.remote())

# depth = 10
# bid_size, bid_price, ask_size, ask_price = ray.get(g_store.get_obs_from_agt.remote(0, depth))

### LOB order imbalance

In [28]:
# ord_imb_store = ord_imb(bid_size, ask_size)
# plot_LOB_subplot(ord_imb_store, depth, '_ord_imb')

### LOB sum of order imbalance

In [29]:
# ord_imb_store = np.asarray(ord_imb_store)
# sum_ord_imb_store = sum_ord_imb(ord_imb_store)
# plot_sum_ord_imb(sum_ord_imb_store, "sum_ord_imb")

### LOB mid price (subplot)

In [30]:
# mid_price_store = mid_price(bid_price, ask_price)
# plot_LOB_subplot(mid_price_store, depth, '_mid_price')

### LOB mid prices

In [31]:
# plot_mid_prices(mid_price_store,)

### LOB bid size

In [32]:
# plot_LOB_subplot(bid_size, depth, '_bid_size')

### LOB ask size

In [33]:
# plot_LOB_subplot(ask_size, depth, '_ask_size')

### LOB bid price

In [34]:
# plot_LOB_subplot(bid_price, depth, '_bid_price')

### LOB ask price

In [35]:
# plot_LOB_subplot(ask_price, depth, '_ask_price')

In [36]:
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_datetime)

2025-08-25 02:04:06
